In [1]:
spark

In [2]:
# !hdfs -h

In [3]:
# !hdfs dfs -ls /cms/users/eatthaph

In [1]:
# !hdfs dfs -ls /cms/users/eatthaph/

In [5]:
!hdfs dfs -ls /project/awg/cms/crab/tasks/2023-07-19

23/07/25 16:06:21 WARN ipc.Client: Exception encountered while connecting to the server 
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.ipc.StandbyException): Operation category READ is not supported in state standby. Visit https://s.apache.org/sbnn-error
	at org.apache.hadoop.security.SaslRpcClient.saslConnect(SaslRpcClient.java:376)
	at org.apache.hadoop.ipc.Client$Connection.setupSaslConnection(Client.java:622)
	at org.apache.hadoop.ipc.Client$Connection.access$2300(Client.java:413)
	at org.apache.hadoop.ipc.Client$Connection$2.run(Client.java:822)
	at org.apache.hadoop.ipc.Client$Connection$2.run(Client.java:818)
	at java.base/java.security.AccessController.doPrivileged(Native Method)
	at java.base/javax.security.auth.Subject.doAs(Subject.java:423)
	at org.apache.hadoop.security.UserGroupInformation.doAs(UserGroupInformation.java:1845)
	at org.apache.hadoop.ipc.Client$Connection.setupIOstreams(Client.java:818)
	at org.apache.hadoop.ipc.Client$Connection.access$3800(Client.

In [1]:
# import pickle
from datetime import datetime, timedelta

# import click
import os
import pandas as pd
# import pprint
import time
# from dateutil.relativedelta import relativedelta
from pyspark import SparkContext, StorageLevel
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, collect_list, concat_ws, greatest, lit, lower, when, unix_timestamp, to_timestamp,
    avg as _avg,
    count as _count,
    hex as _hex,
    max as _max,
    min as _min,
    round as _round,
    sum as _sum,
)

from pyspark.sql.types import (
    LongType,
)

# import matplotlib.pyplot as plt
import numpy as np
import math
import json
#from CMSSpark.src.python.CMSSpark import schemas as cms_schemas

## load dataset

In [34]:
# end_date = str(datetime.now())[:10]
# start_date = str(datetime.now()-timedelta(days=1))[:10]

start_date = '2023-07-20'
end_date = '2023-07-25'

wa_date = end_date
HDFS_CRAB_part = f'/project/awg/cms/crab/tasks/{wa_date}/'
# HDFS_CRAB_part = f'/project/awg/cms/crab/{wa_date}/tasks/'

In [35]:
crab_part = spark.read.format('avro').load(HDFS_CRAB_part)

## Query

In [37]:
df = crab_part.select("TM_TASKNAME","TM_START_TIME","TM_TASK_STATUS","TM_SPLIT_ALGO","TM_USERNAME","TM_USER_ROLE","TM_JOB_TYPE","TM_IGNORE_LOCALITY","TM_SCRIPTEXE","TM_USER_CONFIG")
df.createOrReplaceTempView("crab_algo")
# df.show(10)

In [38]:
query = f"""\
SELECT *
FROM crab_algo 
WHERE 1=1
AND TM_START_TIME >= unix_timestamp("{start_date} 00:00:00", "yyyy-MM-dd HH:mm:ss")*1000 
AND TM_START_TIME < unix_timestamp("{end_date} 00:00:00", "yyyy-MM-dd HH:mm:ss")*1000 
"""

tmpdf = spark.sql(query)
tmpdf.count()

6147

In [21]:
tmpdf.printSchema()

root
 |-- TM_TASKNAME: string (nullable = true)
 |-- TM_START_TIME: long (nullable = true)
 |-- TM_TASK_STATUS: string (nullable = true)
 |-- TM_SPLIT_ALGO: string (nullable = true)
 |-- TM_USERNAME: string (nullable = true)
 |-- TM_USER_ROLE: string (nullable = true)
 |-- TM_JOB_TYPE: string (nullable = true)
 |-- TM_IGNORE_LOCALITY: string (nullable = true)
 |-- TM_SCRIPTEXE: string (nullable = true)
 |-- TM_USER_CONFIG: string (nullable = true)



In [10]:
docs = tmpdf.toPandas().to_dict('records')

In [11]:
len(docs)

6147

In [24]:
# Extract 'REQUIRE_ACCELERATOR' from 'TM_USER_CONFIG'

for i in range(len(docs)):
    if docs[i]['TM_USER_CONFIG'] is not None:
        data = json.loads(docs[i]['TM_USER_CONFIG'])
        if "requireaccelerator" in data:
            docs[i]['REQUIRE_ACCELERATOR'] = data["requireaccelerator"]
        else:
            docs[i]['REQUIRE_ACCELERATOR'] = None
    else:
        docs[i]['REQUIRE_ACCELERATOR'] = None

In [25]:
docs[:3]

[{'TM_TASKNAME': '160406_111833:sciaba_HC-163-AnySite-26725-20160406125703-T1_UK_RAL',
  'TM_START_TIME': 1459934313843,
  'TM_TASK_STATUS': 'SUBMITTED',
  'TM_SPLIT_ALGO': 'FileBased',
  'TM_USERNAME': 'sciaba',
  'TM_USER_ROLE': 'production',
  'TM_JOB_TYPE': 'Analysis',
  'TM_IGNORE_LOCALITY': 'T',
  'TM_SCRIPTEXE': None,
  'TM_USER_CONFIG': None,
  'REQUIRE_ACCELERATOR': None},
 {'TM_TASKNAME': '160406_111914:sciaba_HC-148-AnySite-26727-20160406131903-T2_UK_SGrid_Bristol',
  'TM_START_TIME': 1459934354531,
  'TM_TASK_STATUS': 'SUBMITTED',
  'TM_SPLIT_ALGO': 'FileBased',
  'TM_USERNAME': 'sciaba',
  'TM_USER_ROLE': 'production',
  'TM_JOB_TYPE': 'Analysis',
  'TM_IGNORE_LOCALITY': 'T',
  'TM_SCRIPTEXE': None,
  'TM_USER_CONFIG': None,
  'REQUIRE_ACCELERATOR': None},
 {'TM_TASKNAME': '160319_180958:sciaba_HC-138-AnySite-26052-20160319011302-T2_RU_IHEP',
  'TM_START_TIME': 1458407398241,
  'TM_TASK_STATUS': 'SUBMITTED',
  'TM_SPLIT_ALGO': 'FileBased',
  'TM_USERNAME': 'sciaba',
  'TM_

In [26]:
import osearch

In [27]:
def get_index_schema():
    return {
        "settings": {"index": {"number_of_shards": "1", "number_of_replicas": "1"}},
        "mappings": {
            "properties": {
                "TM_TASKNAME": {"ignore_above": 2048, "type": "keyword"},
                "TM_START_TIME": {"format": "epoch_millis", "type": "date"},
                'TM_TASK_STATUS': {"ignore_above": 2048, "type": "keyword"},
                "TM_SPLIT_ALGO": {"ignore_above": 2048, "type": "keyword"},
                "TM_USERNAME": {"ignore_above": 2048, "type": "keyword"},
                "TM_USER_ROLE": {"ignore_above": 2048, "type": "keyword"},
                "TM_JOB_TYPE": {"ignore_above": 2048, "type": "keyword"},
                "TM_IGNORE_LOCALITY": {"ignore_above": 2048, "type": "keyword"},
                "TM_SCRIPTEXE": {"ignore_above": 2048, "type": "keyword"},
                "REQUIRE_ACCELERATOR": {"ignore_above": 2048, "type": "keyword"},
            }
        }
    }

# def get_index_schema():
#     return {
#         "settings": {"index": {"number_of_shards": "1", "number_of_replicas": "1"}},
#         "mappings": {
#             "properties": {
#                 "TM_TASKNAME": {"ignore_above": 2048, "type": "keyword"},
#                 "TM_SPLIT_ALGO": {"ignore_above": 2048, "type": "keyword"},
#                 "TM_START_TIME": {"format": "epoch_millis", "type": "date"},
#                 "TM_END_INJECTION": {"format": "epoch_millis", "type": "date"},
#             }
#         }
#     }

In [28]:
_index_template = 'crab-data-ekong'
client = osearch.get_es_client("es-cms1.cern.ch/es", 'secret_opensearch.txt', get_index_schema())
# index_mod="": 'test-foo', index_mod="Y": 'test-foo-YYYY', index_mod="M": 'test-foo-YYYY-MM', index_mod="D": 'test-foo-YYYY-MM-DD',
idx = client.get_or_create_index(timestamp=time.time(), index_template=_index_template, index_mod="M")
client.send(idx, docs, metadata=None, batch_size=10000, drop_nulls=False)

/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. S

/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. S

/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. S

/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. S

/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. S

/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. S

0